# Transfer Learning for tabular Data

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs

### Data Preparation

In [3]:
x,y = make_blobs(n_samples=1000,n_features=2,random_state=5,centers=3,cluster_std=2)
x.shape,y.shape

((1000, 2), (1000,))

In [4]:
pd.DataFrame(x)

,0,1
0,-0.243873,-0.202686
1,-4.434212,9.608176
2,-3.483785,8.106750
3,-3.742543,5.224560
4,-7.303895,6.569630
...,...,...
995,-9.435048,11.111028
996,-2.933627,6.038334
997,-5.797862,6.406333
998,-8.151636,8.593986


In [6]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
y.shape

(1000, 3)

In [7]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(x,y,test_size=0.2,random_state=5)
print(xtr.shape,xts.shape,ytr.shape,yts.shape)

(800, 2) (200, 2) (800, 3) (200, 3)


## Modelling the network

In [8]:
from tensorflow.keras import models,layers

In [12]:
ip = layers.Input(shape=(2,))
d1 = layers.Dense(10,activation='relu')(ip)
d2 = layers.Dense(8,activation='relu')(d1)

op = layers.Dense(3,activation='softmax')(d2)

model = models.Model(inputs=ip,outputs=op)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(xtr,ytr,epochs=100,validation_data=(xts,yts))

Epoch 1/100
25/25 [==============================] - 1s 7ms/step - loss: 1.0919 - accuracy: 0.4462 - val_loss: 1.0541 - val_accuracy: 0.4150
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.9615 - accuracy: 0.4975 - val_loss: 0.9650 - val_accuracy: 0.4600
Epoch 3/100
25/25 [==============================] - 0s 3ms/step - loss: 0.8892 - accuracy: 0.5300 - val_loss: 0.9071 - val_accuracy: 0.5350
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 0.8376 - accuracy: 0.5675 - val_loss: 0.8583 - val_accuracy: 0.5550
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.7981 - accuracy: 0.5913 - val_loss: 0.8234 - val_accuracy: 0.5750
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.7693 - accuracy: 0.6062 - val_loss: 0.7933 - val_accuracy: 0.5900
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 0.7435 - accuracy: 0.6087 - val_loss: 0.7750 - val_accuracy: 0.5850
Epoch 8/100
2

In [13]:
model.fit(xtr,ytr,epochs=100,validation_data=(xts,yts))

Epoch 1/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5094 - accuracy: 0.7088 - val_loss: 0.5170 - val_accuracy: 0.7500
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5081 - accuracy: 0.6950 - val_loss: 0.5108 - val_accuracy: 0.7600
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5111 - accuracy: 0.7200 - val_loss: 0.5195 - val_accuracy: 0.7200
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5074 - accuracy: 0.7125 - val_loss: 0.5161 - val_accuracy: 0.7550
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5065 - accuracy: 0.7212 - val_loss: 0.5146 - val_accuracy: 0.7600
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5072 - accuracy: 0.7025 - val_loss: 0.5137 - val_accuracy: 0.7600
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5060 - accuracy: 0.7075 - val_loss: 0.5139 - val_accuracy: 0.7550
Epoch 8/100
2

In [53]:
model.save("model.h5",save_format='tf')

# Transfer Learning

In [54]:
x,y = make_blobs(n_samples=400,n_features=2,random_state=5,centers=5,cluster_std=2)
x.shape,y.shape

((400, 2), (400,))

In [55]:
pd.DataFrame(x)

,0,1
0,-5.900323,8.116902
1,-7.086752,9.421607
2,-4.017509,-5.387742
3,-7.722870,8.301273
4,-5.572526,7.212511
...,...,...
395,-5.139052,8.107650
396,-3.594221,-6.924680
397,4.002661,0.510064
398,-4.948936,8.458345


In [56]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
y.shape

(400, 5)

In [57]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(x,y,test_size=0.2,random_state=5)
print(xtr.shape,xts.shape,ytr.shape,yts.shape)

(320, 2) (80, 2) (320, 5) (80, 5)


In [58]:
#loading the pretrained model
premodel = models.load_model("model.h5")
premodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 27        
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


In [59]:
last_layer = model.layers[-2].output
last_layer

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'dense_10')>

In [60]:
op = layers.Dense(5,activation='softmax')(last_layer)

ValueError: ignored

In [42]:
premodel.get_layer("input_4").input

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'input_4')>

In [44]:
premodel.layers[0].input

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'input_4')>

In [45]:
premodel.input

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'input_4')>

In [63]:
ip = premodel.get_layer("input_4").input

In [64]:
model = models.Model(inputs = ip, outputs=op)
model.summary()

ValueError: ignored